In [1]:
# Import libraries
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
# Download the closing prices
tickers = ['GOOG', 'AMZN', 'TSLA']
stocks = yf.Tickers(tickers)
hist_data = stocks.history(period='1y')['Close']

[*********************100%***********************]  3 of 3 completed


In [3]:
hist_data

,AMZN,GOOG,TSLA
Date,,,
2023-05-22,115.010002,125.870003,188.869995
2023-05-23,114.989998,123.290001,185.770004
2023-05-24,116.750000,121.639999,182.899994
2023-05-25,115.000000,124.349998,184.470001
2023-05-26,120.110001,125.430000,193.169998
...,...,...,...
2024-05-16,183.630005,175.429993,174.839996
2024-05-17,184.699997,177.289993,177.460007
2024-05-20,183.539993,178.460007,174.949997


In [4]:
# Set the portfolio weights (summing up to 1)
weights = np.array([0.2, 0.3, 0.5])

In [5]:
# Set the initial investment
initial_investment = 100000

In [6]:
# Compute the daily return rates
return_rates = hist_data.pct_change()
return_rates = return_rates.dropna()
return_rates

,AMZN,GOOG,TSLA
Date,,,
2023-05-23,-0.000174,-0.020497,-0.016413
2023-05-24,0.015306,-0.013383,-0.015449
2023-05-25,-0.014989,0.022279,0.008584
2023-05-26,0.044435,0.008685,0.047162
2023-05-30,0.012905,-0.006298,0.041363
...,...,...,...
2024-05-16,-0.012689,0.008914,0.004885
2024-05-17,0.005827,0.010603,0.014985
2024-05-20,-0.006280,0.006599,-0.014144


In [7]:
# Compute the mean of return rates
mean_return_rates = return_rates.mean()
mean_return_rates

AMZN    0.002022
GOOG    0.001515
TSLA    0.000343
dtype: float64

In [8]:
# Generate the variance-covariance matrix
cov_matrix = return_rates.cov()
cov_matrix

,AMZN,GOOG,TSLA
AMZN,0.000321,0.000169,0.000145
GOOG,0.000169,0.000313,0.000102
TSLA,0.000145,0.000102,0.001046


In [9]:
# Compute the mean of portfolio r.r.
mean_port = mean_return_rates.dot(weights)
mean_port

0.0010305902764093135

In [10]:
# Compute the standard deviation of portfolio r.r.
std_port = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
std_port

0.0195465938519163

In [11]:
# Compute the z-value
from scipy.stats import norm
alpha = 0.95
z_value = norm.ppf(1 - alpha)
z_value

-1.6448536269514722

In [12]:
# Compute the (1 - alpha)th quantile
quantile = mean_port + std_port * z_value
quantile

-0.031120695515462555

In [13]:
# Compute the daily VaR of the portfolio
VaR_port = - initial_investment * quantile
VaR_port

3112.0695515462553

In [14]:
# Define a function to compute the value with a k-day horizon
def VaR_days(k, VaR_port):
    return np.sqrt(k) * VaR_port

In [15]:
# Call the function to compute the VaR on a 2-day horizon
VaR_days(2, VaR_port)

4401.13096684507

In [16]:
# Extract a subset of columns of the DataFrame
garr = return_rates[['GOOG', 'AMZN']]

In [17]:
garr

,GOOG,AMZN
Date,,
2023-05-23,-0.020497,-0.000174
2023-05-24,-0.013383,0.015306
2023-05-25,0.022279,-0.014989
2023-05-26,0.008685,0.044435
2023-05-30,-0.006298,0.012905
...,...,...
2024-05-16,0.008914,-0.012689
2024-05-17,0.010603,0.005827
2024-05-20,0.006599,-0.006280


In [18]:
# Save the extracted DataFrame not saving the index as a new column
garr.to_csv('GARR.csv', index=False)